# Testing a pure pytorch MPC design

In [ ]:
import torch as th, matplotlib.pyplot as plt, numpy as np
π = th.pi
from torch.nn import Module, Parameter
import torch.nn.functional as F
from tqdm import tqdm

In [ ]:
from matplotlib.ticker import MultipleLocator, FuncFormatter
# define model dynamics, single pendulum, input = torque
def pendulum_dynamics(x, u):
    # x = [theta, theta_dot]
    # u = [tau] (torque)
    g = 9.81
    l = 1.0
    m = 1.0
    b = 0.1 # damping
    dtheta = x[1]
    ddtheta = (u[0] - m * g * l * th.sin(x[0]) - b * x[1]) / (m * l**2)
    return th.stack([dtheta, ddtheta])

def rk4_step(func, x, u, dt):
    k1 = func(x, u)
    k2 = func(x + 0.5 * dt * k1, u)
    k3 = func(x + 0.5 * dt * k2, u)
    k4 = func(x + dt * k3, u)
    return x + (dt / 6) * (k1 + 2 * k2 + 2 * k3 + k4)

def euler_step(func, x, u, dt):
    k1 = func(x, u)
    return x + dt * k1

def anim(xs):
    from matplotlib.animation import FuncAnimation
    from IPython.display import HTML, display
    fig, ax = plt.subplots()
    line, = ax.plot([], [], 'o-')
    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    ax.set_aspect('equal')

    def update(frame):
        x = xs[frame]
        line.set_data([0, th.cos(x[0]+π/2)], [0, th.sin(x[0]+π/2)])
        return line,

    ani = FuncAnimation(fig, update, frames=len(xs), blit=True, interval=3*100/6)
    plt.close(fig)
    return display(HTML(ani.to_jshtml()))

def my_plot(xs):
    # plot trajectory in phase space
    plt.figure(figsize=(8, 4))
    plt.plot(xs[:, 0].numpy(), xs[:, 1].numpy())
    def pi_formatter(x, pos):
        frac = x / np.pi
        if np.isclose(frac, 0):
            return "0"
        elif np.isclose(frac, 0.5):
            return r"$\frac{\pi}{2}$"
        elif np.isclose(frac, 1):
            return r"$\pi$"
        elif np.isclose(frac, -0.5):
            return r"$-\frac{\pi}{2}$"
        elif np.isclose(frac, -1):
            return r"$-\pi$"
        else:
            return r"${:.2f}\pi$".format(frac)

    plt.gca().xaxis.set_major_locator(MultipleLocator(π/8))
    plt.gca().xaxis.set_major_formatter(FuncFormatter(pi_formatter))
    plt.xlabel('Theta (rad)')
    plt.ylabel('Omega (rad/s)')
    plt.title('Phase Space Trajectory')

    plt.tight_layout()
    plt.show()

    anim(xs[::10]);

In [ ]:
## Create a simulation
x0 = th.tensor([0.5, 0.0])  # initial state (theta, omega)
dt = 0.01  # time step
T = 10.0  # total time

num_steps = int(T / dt)
xs = th.zeros((num_steps + 1, 2))
xs[0] = x0
for i in range(num_steps):
    u = th.tensor([1.0])  # constant torque
    x = rk4_step(pendulum_dynamics, xs[i], u, dt)
    xs[i + 1] = x

my_plot(xs[::3])

In [ ]:
import torch as th
import torch.nn.functional as F
from torch.nn import Parameter

def calc_optimal_u(x0, H, n_iter=10, dt=0.01, verbose=False):
    us = Parameter(th.zeros(H, 1), requires_grad=True)  # initialize control inputs
    lrs = 0.5 * th.logspace(0, -2, n_iter)
    # optimizer = th.optim.Adam([us], lr=lrs[0])
    optimizer = th.optim.AdamW([us], lr=lrs[0], weight_decay=1e-2)

    for i in range(n_iter):  # optimization loop
        optimizer.zero_grad()
        for pg in optimizer.param_groups: pg['lr'] = lrs[i]  # update learning rate

        # Initialize xs as a list to append states
        states = [x0]

        for t in range(H):
            u = us[t]
            # next_x = rk4_step(pendulum_dynamics, states[-1], u, dt)
            next_x = euler_step(pendulum_dynamics, states[-1], u, dt)
            states.append(next_x)
        
        # Stack the list of states into a single tensor
        xs = th.stack(states)

        state_loss = th.sum(xs[:, 0]**2) + 0.1 * th.sum(xs[:, 1]**2)  # target: theta=0, omega=0
        control_loss = 1e-2 * th.sum(th.diff(xs, dim=0)**2) + 1e-5 * th.sum(us**2)

        loss = state_loss + control_loss
        loss.backward()
        optimizer.step()

        if verbose: print(f'Iter: {i+1}/{n_iter}, Loss: {loss.item()}')
        u0 = us[0].detach()
    # print(f'Optimal control inputs: {u0}, loss: {loss.item()}')
    return u0, loss.item()

# test
u, _ = calc_optimal_u(th.tensor([0.5, 0.0]), 10, verbose=True)
print(u)

In [ ]:
## Test Adam optimizer on input
H = 10 # horizon length [steps]
dt = 0.01  # time step [s]
T = 10.0  # total time [s]

n_sim_steps = int(T / dt)

x0 = th.tensor([0.3, 0.0])  # initial state (theta, omega)

# simulation with optimal control inputs at every step
xs = th.zeros((n_sim_steps + 1, 2))
xs[0] = x0
us, losses = th.zeros(n_sim_steps), th.zeros(n_sim_steps)
for t in tqdm(range(n_sim_steps)):
    u, losses[t] = calc_optimal_u(xs[t], H)
    xs[t + 1] = rk4_step(pendulum_dynamics, xs[t], u, dt) + th.randn(2)*0.002
    us[t] = u

In [ ]:
#plot xs and u on a 3 raws plot
plt.figure(figsize=(12, 8))

plt.subplot(3, 1, 1)
plt.plot(xs[:, 0].numpy(), label='Theta (rad)')
plt.plot(xs[:, 1].numpy(), label='Omega (rad/s)')
plt.ylabel('States')
plt.legend()

plt.subplot(3, 1, 2)
plt.plot(us.numpy(), label='Control Inputs (Torque)')
plt.ylabel('Control Inputs')
plt.legend()

plt.subplot(3, 1, 3)
plt.plot(losses.numpy(), label='Loss')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

my_plot(xs)